In [1]:
from MancalaBoard import MancalaBoard
from Game import Game
from Play import Play
import pygame
import sys

pygame 2.6.1 (SDL 2.28.4, Python 3.10.14)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:

# Initialize Pygame
pygame.init()

# Constants
SCREEN_WIDTH, SCREEN_HEIGHT = 800, 700
BACKGROUND_COLOR = (60, 80, 34)  # Green background
PIT_COLOR = (210, 180, 140)  # Light brown
STORE_COLOR = (139, 69, 19)  # Dark brown
TEXT_COLOR = (255, 255, 255)  # White
FONT = pygame.font.Font(None, 36)

# Dimensions
PIT_RADIUS = 40
STORE_WIDTH, STORE_HEIGHT = 60, 300
MARGIN = 20

# Initialize screen
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Mancala Game")

# Function to draw the board
# Function to draw the board
def draw_board(board_state, human_pits):
    screen.fill(BACKGROUND_COLOR)

    pits_positions = {}
    for key, value in board_state.items():
        if key in [1, 2]:  # Draw stores
            x = MARGIN if key == 1 else SCREEN_WIDTH - MARGIN - STORE_WIDTH
            y = (SCREEN_HEIGHT - STORE_HEIGHT) // 2
            pygame.draw.rect(screen, STORE_COLOR, (x, y, STORE_WIDTH, STORE_HEIGHT))
            draw_text(str(value), x + STORE_WIDTH // 2, y + STORE_HEIGHT // 2)
        elif isinstance(key, str):  # Draw pits
            # Check if the current pit belongs to the human player
            if key in human_pits:  # Human player's pits (bottom row)
                row = 1  # Place at the bottom row
            else:  # Opponent player's pits
                row = 0  # Place at the top row

            # Adjust the positions for each row and pit
            if row == 0:  # Opponent pits (top row)
                col = ord(key) - ord('G') if key in game.state.player2_pits else ord(key) - ord('A')
                x = 3 * MARGIN + col * (2 * PIT_RADIUS + MARGIN)
                y = MARGIN
            else:  # Human pits (bottom row)
                col = ord(key) - ord('A') if key in game.state.player1_pits else ord(key) - ord('G')
                x = 3 * MARGIN + col * (2 * PIT_RADIUS + MARGIN)
                y = SCREEN_HEIGHT - MARGIN - 2 * PIT_RADIUS

            pygame.draw.circle(screen, PIT_COLOR, (x + PIT_RADIUS, y + PIT_RADIUS), PIT_RADIUS)
            text = f"{key} = {value}"
            draw_text(text, x + PIT_RADIUS, y + PIT_RADIUS)

            # Store pit positions for click detection
            pits_positions[key] = (x + PIT_RADIUS, y + PIT_RADIUS)

    return pits_positions


# Function to draw text on the screen
def draw_text(text, x, y):
    text_surface = FONT.render(text, True, TEXT_COLOR)
    text_rect = text_surface.get_rect(center=(x, y))
    screen.blit(text_surface, text_rect)

# Check if a pit was clicked
def check_click(pos, pits_positions):
    for pit, center in pits_positions.items():
        distance = ((pos[0] - center[0]) ** 2 + (pos[1] - center[1]) ** 2) ** 0.5
        if distance <= PIT_RADIUS:  # Click is within the pit's circle
            return pit
    return None

# Initial board state
board_state = {
    'A': 4, 'B': 4, 'C': 4, 'D': 4, 'E': 4, 'F': 4,  # Player 1 pits
    'G': 4, 'H': 4, 'I': 4, 'J': 4, 'K': 4, 'L': 4,  # Player 2 pits
    '1': 0, '2': 0,  # Stores
}
def choose_player_side():
    pygame.init()
    screen = pygame.display.set_mode((800, 700))
    pygame.display.set_caption("Choose Player Side")
    font = pygame.font.Font(None, 74)

    running = True
    player_side = None

    while running:
        screen.fill((0, 0, 0))
        text1 = font.render("Press 1 for Player 1", True, (255, 255, 255))
        text2 = font.render("Press 2 for Player 2", True, (255, 255, 255))

        screen.blit(text1, (200, 200))
        screen.blit(text2, (200, 300))

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_1:
                    player_side = 1
                    running = False
                elif event.key == pygame.K_2:
                    player_side = 2
                    running = False

        pygame.display.flip()

    return player_side


def draw_game_over_screen(winner, score):
    screen.fill(BACKGROUND_COLOR)
    text = f"Game Over! Winner: {winner} with score {score}"
    draw_text(text, SCREEN_WIDTH // 2, SCREEN_HEIGHT // 3)

    restart_text = "Press 'R' to restart or 'Q' to quit."
    draw_text(restart_text, SCREEN_WIDTH // 2, SCREEN_HEIGHT // 2)

    pygame.display.flip()

# Game loop
def main():
    pygame.init()
    board = MancalaBoard()
    game = Game(board, player_side)

    play = Play(game)

    clock = pygame.time.Clock()
    game_over = False

    # Determine which pits belong to the human player
    human_pits = game.state.player1_pits if game.player_side["HUMAN"] == 1 else game.state.player2_pits

    while True:
        if not game_over:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()

                if event.type == pygame.MOUSEBUTTONDOWN:
                    pits_positions = draw_board(game.state.board, human_pits)
                    clicked_pit = check_click(event.pos, pits_positions)

                    if clicked_pit and clicked_pit in (game.state.player1_pits if chosen_side == 1 else game.state.player2_pits):
                        # Human turn
                        game_over = play.human_turn(clicked_pit)

                        # Computer turn if game not over
                        if not game_over:
                            game_over = play.computer_turn()

            # Update the display if the game is not over
            if not game_over:
                pits_positions = draw_board(game.state.board, human_pits)
                pygame.display.flip()

        else:
            # Game over: Draw game over screen and handle restart or quit
            winner, score = game.find_winner()
            draw_game_over_screen(winner, score)

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_r:

                        main()  # Restart by calling the main function
                    elif event.key == pygame.K_q:  # Quit game
                        pygame.quit()
                        sys.exit()

        clock.tick(30)


if __name__ == "__main__":
    chosen_side = choose_player_side()
    print(f"You chose Player {chosen_side}")
    board = MancalaBoard()
    player_side = {"HUMAN": chosen_side, "COMPUTER": 1 if chosen_side == 2 else 2}
    game = Game(board, player_side)
    play = Play(game)
    main()


You chose Player 1
Clicked on pit: C
Computer's turn
Computer chose pit: I
Clicked on pit: E
Computer's turn
Computer chose pit: J
Clicked on pit: F
Computer's turn
Computer chose pit: H
Clicked on pit: B
Computer's turn
Computer chose pit: G
Clicked on pit: F
Computer's turn
Computer chose pit: L


SystemExit: 